## Candidates identification and keywords extraction
Directions and resources: http://bdewilde.github.io/blog/2014/09/23/intro-to-automatic-keyphrase-extraction/

Methodology of automatic keyphrase extraction:

1. Identify: a set of words and phrases that could convey the topical content of a document are identified.

2. Extract: candidates are scored/ranked and the “best” are selected as a document’s keyphrases.


__Candidate Identification__

* remove stop words and punctuation; 
* filter for words with certain parts of speech;
* (or) filter for multi-word phrases, certain POS patterns;
* use external knowledge bases like WordNet or Wikipedia as a reference source of good/bad keyphrases.

__Options__

* take all of the n-grams (where 1 ≤ n ≤ 5)
* limit candidates to only "noun phrases" matching the POS pattern '{(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}' (a regular expression written in a simplified format used by NLTK’s RegexpParser()). This matches any number of adjectives followed by at least one noun that may be joined by a preposition to one other adjective(s)+noun(s) sequence.

__Keyphrases extraction__

1. Unsupervised Algorithms
    * no "training data" needed
    * graph-based ranking method: document = network with nodes as candidate keyphrases and whose edges (optionally weighted by the degree of relatedness) connect related candidates. Implementation: Graph-based ranking algorithm.
    

2. Supervised Algorithms
    * need text with already-labeled examples (“training data”)
    * Two primary developments deal with __task reformulation__ and __feature design__

##### Task Refolmulation

__Binary classification problem__: some fraction of candidates are classified as keyphrases and the rest as non-keyphrases. Methods: Naive Bayes, decision trees, and support vector machines. Problems: this reformulation of the task is conceptually problematic; humans judge keyphrases in relative sense (not independent from one another).<br>
Implementation - KEA (as published in Practical Automatic Keyphrase Extraction), used TF*IDF and position of first occurrence (while filtering on phrase length) to identify keyphrases

__Ranking problem__: a function is trained to rank candidates pairwise according to degree of “keyness”. The best candidates rise to the top, and the top N are taken to be the document’s keyphrases.<br> Implementation - Linear Ranking SVM to rank candidate keyphrases. 

###### Feature design

* __frequency statistics + other statistical features__: phrase length (number of constituent words), phrase position (normalized position within a document of first and/or last occurrence therein), and “supervised keyphraseness” (number of times a keyphrase appears as such in the training data). 
* __structural features__: titles, abstracts, intros and conclusions, metadata etc.
* __external resource-based features__: “Wikipedia-based keyphraseness” assumes that keyphrases are more likely to appear as Wiki article links and/or titles.
* __phrase commonness__: compare a candidate’s frequency in a document with respect to its frequency in an external corpus.

In [1]:
import os, sys, re, csv
import heapq
import string
import gensim
import itertools
import json
from operator import itemgetter
import nltk
from nltk import *
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [2]:
# Grab corpus of documents (created earlier, not shown)
corpus_dir = nltk.data.find('/Users/dariaulybina/Desktop/georgetown/ml_practice/imf_direct/corpus/')
my_corpus= nltk.corpus.PlaintextCorpusReader(corpus_dir, '.*\.txt')
fids = my_corpus.fileids()
example = my_corpus.raw(fileids = 'p10035.txt')

In [3]:
def get_meta():
    path1 = '/Users/dariaulybina/Desktop/georgetown/ml_practice/imf_direct/metadata_blogs.json'
    with open(path1) as datafile:
        data = json.load(datafile)
        datafile.close()
    return data

In [4]:
# saving metadata to dictionary to use later for search methods:
# structure: {'filename.txt':{'date':'x/x/xxxx','title':'xxx'}}
data = get_meta()
dict_main = {}
for d in data:
    naming = d['id']+'.txt'
    dict1 = {naming:{'date':d['date'],'title':d['title']}}
    dict_main.update(dict1)

In [5]:
print(dict_main['p10035.txt'])

{'date': '7/22/2015', 'title': 'From Windfall to Windmill: Harnessing Asiaâ€\x99s Dynamism for Latin America'}


In [13]:
print(fids[15:30])
print("Total number of posts: {}".format(len(fids)))

['p10237.txt', 'p1027.txt', 'p10271.txt', 'p10292.txt', 'p10314.txt', 'p10329.txt', 'p10344.txt', 'p10347.txt', 'p10353.txt', 'p10392.txt', 'p10396.txt', 'p10417.txt', 'p10441.txt', 'p10460.txt', 'p10473.txt']
Total number of posts: 796


In [10]:
# Option 1
def extract_candidate_chunks(text, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'):
    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    # join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group).lower()
                  for key, group in itertools.groupby(all_chunks, lambda word__pos__chunk: word__pos__chunk[2] != 'O') if key]
    return [cand for cand in candidates if cand not in stop_words and not all(char in punct for char in cand)]


# Option 2: Original TextRank algorithm - extracting all (unigram) nouns and adjectives
def extract_candidate_words(text, good_tags=set(['JJ','JJR','JJS','NN','NNP','NNS','NNPS'])):
	# exclude candidates that are stop words or entirely punctuation
	punct = set(string.punctuation)
	stop_words = set(nltk.corpus.stopwords.words('english'))
	# tokenize and POS-tag words
	tagged_words = itertools.chain.from_iterable(nltk.pos_tag_sents(nltk.word_tokenize(sent)
		for sent in nltk.sent_tokenize(text)))
		# filter on certain POS tags and lowercase all words
	candidates = [word.lower() for word, tag in tagged_words if tag in good_tags and word.lower() not in stop_words and not all(char in punct for char in word)]
	return candidates

#Keyphrase selection - frequency statistic-based approach with gensim (another option - with skilearn)
# Replace candidates='chunks' with 'words' to compare outputs
def score_keyphrases_by_tfidf(texts, candidates):
    # extract candidates from each text in texts, either chunks or words
    extract = {
        'chunks': extract_candidate_chunks,
        'words': extract_candidate_words,
    }[candidates]

    boc_texts = [
        extract(texts.raw(fileid)) for fileid in texts.fileids()
    ]

    # make gensim dictionary and corpus
    dictionary = gensim.corpora.Dictionary(boc_texts)
    corpus = [dictionary.doc2bow(boc_text) for boc_text in boc_texts]

    # transform corpus with tf*idf model
    tfidf = gensim.models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]

    return corpus_tfidf, dictionary

In [8]:
tfidfs, id2word = score_keyphrases_by_tfidf(my_corpus, 'words')

In [9]:
print(type(id2word))
print(id2word)

<class 'gensim.corpora.dictionary.Dictionary'>
Dictionary(19147 unique tokens: ['fisco', "illusions_'the", 'introduce', 'multi-pronged', 'unions']...)


__ISSUE:__ 'introduce' - verb?

In [11]:
tfidfs, id2word = score_keyphrases_by_tfidf(my_corpus, 'chunks')

In [13]:
print(type(tfidfs))
print(tfidfs)
print(type(id2word))
print(id2word)

<class 'gensim.interfaces.TransformedCorpus'>
<class 'gensim.corpora.dictionary.Dictionary'>
Dictionary(54678 unique tokens: ['much room for error', 'unique features', 'jpy per hour', 'remainder', 'past similar circumstances']...)


### TO DO: clean text in corpus:
1. encodings
2. [
3. *
4. by [ 
5. other grammar/typos/'soundcloud'

In [23]:
#Print top 10 keywords by TF-IDF per document
for idx, doc in enumerate(tfidfs):
    print("Document '{}' key phrases:".format(fids[idx]))
    #Get top 10 terms by TF-IDF score
    for wid, score in heapq.nlargest(10, doc, key=itemgetter(1)):
        print("{:0.3f}: {}".format(score, id2word[wid]))
    print()

Document 'p10016.txt' key phrases:
0.351: week imf economists
0.351: lead author in [
0.351: oil prices for consumers
0.351: big story
0.351: [ issues
0.351: [ imfdirect remember
0.225: exporters
0.221: [ new paper
0.221: oil importers
0.209: podcast

Document 'p1002.txt' key phrases:
0.306: aml/cft
0.203: financing of terrorism
0.171: trafficking
0.153: fsaps
0.153: macroeconomic impact
0.135: money laundering
0.135: terrorist financing
0.119: fund
0.085: corresponding increase in resources
0.085: madoff scandals

Document 'p10035.txt' key phrases:
0.248: latin america
0.142: asia
0.111: trade integration
0.085: example of successful regional trade integration
0.085: notable exception of mexico
0.085: linchpin of large cross- border trade
0.085: array of supply-side bottlenecks
0.085: broad set of manufactured goods
0.085: weak outlook for growth
0.085: links with asia

Document 'p10050.txt' key phrases:
0.406: demographic transition
0.244: demographic dividend
0.138: female labor for

#### Document 'p10035.txt' key phrases with TF-IDF:
0.248: latin america <br>
0.142: asia <br>
0.111: trade integration <br>
0.085: example of successful regional trade integration <br>
0.085: notable exception of mexico <br>
0.085: linchpin of large cross- border trade <br>
0.085: array of supply-side bottlenecks <br>
0.085: broad set of manufactured goods <br>
0.085: weak outlook for growth <br>
0.085: links with asia <br>

#### document 'p10035.txt' metadata: 
date: 7/22/2015 <br>
title: From Windfall to Windmill: Harnessing Asiaâ€\x99s Dynamism for Latin America <br>

### Text Rank Algorithm

Importance of a candidate is determined by its relatedness to other candidates, where “relatedness” may be measured by two terms’ frequency of co-occurrence or semantic relatedness. Method assumes that more important candidates are related to a greater number of other candidates, and that more of those related candidates are also considered important; it does not, however, ensure that selected keyphrases cover all major topics, although multiple variations try to compensate for this weakness.

Essentially, a document is represented as a network whose nodes are candidate keyphrases (typically only key words) and whose edges (optionally weighted by the degree of relatedness) connect related candidates. Then, a graph-based ranking algorithm, such as Google’s famous PageRank, is run over the network, and the highest-scoring terms are taken to be the document’s keyphrases.

* __TextRank__
* __DivRank__: attempts to ensure good topic coverage
* __Topic-based clustering method__

In [14]:
# Text Rank Algorithm
def score_keyphrases_by_textrank(text, n_keywords=0.05):
    from itertools import takewhile, tee
    import networkx, nltk
    
    # tokenize for all words, and extract *candidate* words
    words = [word.lower()
             for sent in nltk.sent_tokenize(text)
             for word in nltk.word_tokenize(sent)]
    candidates = extract_candidate_words(text)
    # build graph, each node is a unique candidate
    graph = networkx.Graph()
    graph.add_nodes_from(set(candidates))
    # iterate over word-pairs, add unweighted edges into graph
    def pairwise(iterable):
        """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
    for w1, w2 in pairwise(candidates):
        if w2:
            graph.add_edge(*sorted([w1, w2]))
    # score nodes using default pagerank algorithm, sort by score, keep top n_keywords
    ranks = networkx.pagerank(graph)
    if 0 < n_keywords < 1:
        n_keywords = int(round(len(candidates) * n_keywords))
    word_ranks = {word_rank[0]: word_rank[1]
                  for word_rank in sorted(ranks.items(), key=lambda x: x[1], reverse=True)[:n_keywords]}
    keywords = set(word_ranks.keys())
    # merge keywords into keyphrases
    keyphrases = {}
    j = 0
    for i, word in enumerate(words):
        if i < j:
            continue
        if word in keywords:
            kp_words = list(takewhile(lambda x: x in keywords, words[i:i+10]))
            avg_pagerank = sum(word_ranks[w] for w in kp_words) / float(len(kp_words))
            keyphrases[' '.join(kp_words)] = avg_pagerank
            # counter as hackish way to ensure merged keyphrases are non-overlapping
            j = i + len(kp_words)
    
    return sorted(keyphrases.items(), key=lambda x: x[1], reverse=True)

def extract_candidate_features(candidates, doc_text, doc_excerpt, doc_title):
    import collections, math, nltk, re
    
    candidate_scores = collections.OrderedDict()
    
    # get word counts for document
    doc_word_counts = collections.Counter(word.lower()
                                          for sent in nltk.sent_tokenize(doc_text)
                                          for word in nltk.word_tokenize(sent))
    
    for candidate in candidates:
        
        pattern = re.compile(r'\b'+re.escape(candidate)+r'(\b|[,;.!?]|\s)', re.IGNORECASE)
        
        # frequency-based
        # number of times candidate appears in document
        cand_doc_count = len(pattern.findall(doc_text))
        # count could be 0 for multiple reasons; shit happens in a simplified example
        if not cand_doc_count:
            print('**WARNING: {} not found!'.format(candidate))
            continue
    
        # statistical
        candidate_words = candidate.split()
        max_word_length = max(len(w) for w in candidate_words)
        term_length = len(candidate_words)
        # get frequencies for term and constituent words
        sum_doc_word_counts = float(sum(doc_word_counts[w] for w in candidate_words))
        try:
            # lexical cohesion doesn't make sense for 1-word terms
            if term_length == 1:
                lexical_cohesion = 0.0
            else:
                lexical_cohesion = term_length * (1 + math.log(cand_doc_count, 10)) * cand_doc_count / sum_doc_word_counts
        except (ValueError, ZeroDivisionError) as e:
            lexical_cohesion = 0.0
        
        # positional
        # found in title, key excerpt
        in_title = 1 if pattern.search(doc_title) else 0
        in_excerpt = 1 if pattern.search(doc_excerpt) else 0
        # first/last position, difference between them (spread)
        doc_text_length = float(len(doc_text))
        first_match = pattern.search(doc_text)
        abs_first_occurrence = first_match.start() / doc_text_length
        if cand_doc_count == 1:
            spread = 0.0
            abs_last_occurrence = abs_first_occurrence
        else:
            for last_match in pattern.finditer(doc_text):
                pass
            abs_last_occurrence = last_match.start() / doc_text_length
            spread = abs_last_occurrence - abs_first_occurrence

        candidate_scores[candidate] = {'term_count': cand_doc_count,
                                       'term_length': term_length, 'max_word_length': max_word_length,
                                       'spread': spread, 'lexical_cohesion': lexical_cohesion,
                                       'in_excerpt': in_excerpt, 'in_title': in_title,
                                       'abs_first_occurrence': abs_first_occurrence,
                                       'abs_last_occurrence': abs_last_occurrence}

    return candidate_scores


### Score keyphrases by textrank
__score_keyphrases_by_textrank__ function is one of the two implementations of the TextRank algorithm. Only unigram candidates (not chunks or n-grams) are added to the network as nodes, the co-occurrence window size is fixed at 2 (so only adjacent words are said to “co-occur”), and the edges between nodes are unweighted (rather than weighted by the number of co-occurrences). The N top-scoring candidates are taken to be its keywords; sequences of adjacent keywords are merged to form key phrases and their individual PageRank scores are averaged, so as not to bias for longer keyphrases.

In [15]:
# earlier I assigned text of random post 'p10035.txt' to give an example of a variable
score_keyphrases_by_textrank(example)

[('trade', 0.023935744809250144),
 ('economic', 0.02085530814649411),
 ('asia', 0.019021875250441186),
 ('trade integration', 0.017248473203482505),
 ('trade relations', 0.015217025918449563),
 ('greater trade integration', 0.014080668776840208),
 ('latin america', 0.01406448257200328),
 ('economic relations', 0.013676807587071547),
 ('greater economic integration', 0.013053856555921528),
 ('growth', 0.012365616288405422),
 ('investment', 0.011760171300803226),
 ('latin american', 0.010788904724167936),
 ('integration', 0.010561201597714868),
 ('structural investment', 0.008915651327254713),
 ('gains', 0.008773336360606149),
 ('development', 0.008376966366639978),
 ('region', 0.008071588335859895),
 ('greater', 0.007745059923555606),
 ('china', 0.007210414961573706),
 ('source', 0.007183335932633898),
 ('political', 0.007123680698616596),
 ('important', 0.006893984223002439),
 ('weak', 0.006793367273639482),
 ('relations', 0.006498307027648983),
 ('american', 0.006226010244993653),
 ('

### Extraction of candidate features - specific features of a keyphrase

In [31]:
candidates = ["trade integration"]
doc_text = example
doc_title  = dict_main['p10035.txt']['title']
doc_excerpt = 'None'
candidate_scores = extract_candidate_features(candidates, doc_text, doc_excerpt, doc_title)
print("Document: p10035.txt")
print("Title: {}".format(doc_title))
print('Term of interest: "trade integration" ')
print(candidate_scores)

Document: p10035.txt
Title: From Windfall to Windmill: Harnessing Asiaâ€s Dynamism for Latin America
Term of interest: "trade integration" 
OrderedDict([('trade integration', {'abs_first_occurrence': 0.10594546267322306, 'spread': 0.7793175383698406, 'in_excerpt': 0, 'term_length': 2, 'max_word_length': 11, 'in_title': 0, 'term_count': 5, 'abs_last_occurrence': 0.8852630010430637, 'lexical_cohesion': 0.6534500016676995})])
